<a href="https://colab.research.google.com/github/Shrutika-TechSavvy/Google-Colab-Codes/blob/main/Matrix_Multiplication_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile matrix_mul.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 4  // Size of the square matrices

int main() {
int A[N][N], B[N][N], C[N][N];
int i, j, k;

// Initialize matrices A and B
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        A[i][j] = i + j;
        B[i][j] = i - j;
        C[i][j] = 0;
    }
}

// Matrix multiplication using OpenMP
#pragma omp parallel for private(i,j,k) shared(A,B,C)
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        int sum = 0;
        for (k = 0; k < N; k++) {
            sum += A[i][k] * B[k][j];
        }
        C[i][j] = sum;
    }
}

// Print the result
printf("Result matrix C:\n");
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        printf("%d ", C[i][j]);
    }
    printf("\n");
}

return 0;

}


Writing matrix_mul.c


In [ ]:

!gcc -fopenmp matrix_mul.c -o matrix_mul
!./matrix_mul


Result matrix C:
14 8 2 -4 
20 10 0 -10 
26 12 -2 -16 
32 14 -4 -22 


In [9]:
%%writefile matrix_mul1.
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

int main() {

    int N;
    printf("Enter matrix size N for NxN matrix: ");
    scanf("%d", &N);

    int size = N * N * sizeof(int);

    // Dynamic memory allocation
    int *A = (int*)malloc(size);
    int *B = (int*)malloc(size);
    int *C = (int*)malloc(size);

    printf("\nEnter %d elements for Matrix A:\n", N*N);
    for(int i = 0; i < N*N; i++)
        scanf("%d", &A[i]);

    printf("\nEnter %d elements for Matrix B:\n", N*N);
    for(int i = 0; i < N*N; i++)
        scanf("%d", &B[i]);

    // Get system CPU information
    int total_cores = omp_get_num_procs();
    printf("\nTotal CPU Cores Available: %d\n", total_cores);

    // Let program use all available cores
    omp_set_num_threads(total_cores);

    // Confirm how many threads OpenMP actually uses
    int used_threads;
    #pragma omp parallel
    {
        #pragma omp single
        used_threads = omp_get_num_threads();
    }

    printf("Threads used by this program: %d\n", used_threads);

    // Start timer for parallel multiplication
    double start_time = omp_get_wtime();

    // Parallel matrix multiplication
    #pragma omp parallel for collapse(2)
    for (int i = 0; i < N; i++) {
        for(int j = 0; j < N; j++) {
            int sum = 0;
            for(int k = 0; k < N; k++) {
                sum += A[i*N + k] * B[k*N + j];
            }
            C[i*N + j] = sum;
        }
    }

    double end_time = omp_get_wtime();
    double omp_time = (end_time - start_time) * 1000.0; // in milliseconds

    // Display the result matrix
    printf("\nResult Matrix C (A x B):\n");
    for(int i = 0; i < N*N; i++) {
        printf("%d ", C[i]);
        if((i+1) % N == 0)
            printf("\n");
    }

    // Show performance data
    printf("\nOpenMP Execution Time: %.4f ms\n", omp_time);
    printf("CPU Core Utilization Efficiency: %.2f%%\n",
           ((double)used_threads / total_cores) * 100.0);

    // Free allocated memory
    free(A); free(B); free(C);

    return 0;
}


Writing matrix_mul1.


In [12]:

!gcc -fopenmp matrix_mul1.c -o matrix_mul1
!./matrix_mul1


matrix_mul1.c:82:27: error: initializer element is not constant
   82 |     double end_parallel = omp_get_wtime();
      |                           ^~~~~~~~~~~~~
matrix_mul1.c:83:44: error: ‘start_parallel’ undeclared here (not in a function); did you mean ‘end_parallel’?
   83 |     double parallel_time = (end_parallel - start_parallel) * 1000.0;
      |                                            ^~~~~~~~~~~~~~
      |                                            end_parallel
matrix_mul1.c:85:12: error: expected declaration specifiers or ‘...’ before string constant
   85 |     printf("\nRESULT (only first 10 elements shown):\n");
      |            ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
matrix_mul1.c:86:5: error: expected identifier or ‘(’ before ‘for’
   86 |     for(int i = 0; i < (N*N < 10 ? N*N : 10); i++)
      |     ^~~
matrix_mul1.c:86:22: error: expected ‘=’, ‘,’, ‘;’, ‘asm’ or ‘__attribute__’ before ‘<’ token
   86 |     for(int i = 0; i < (N*N < 10 ? N*N : 10); i++)
  